In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
#################################################################
def Vh2V3k(el,b, Vh):
  
    alpha = np.pi / 180.
    cosb = np.cos(b*alpha)
    sinb = np.sin(b*alpha)
    cosl = np.cos(el*alpha)
    sinl = np.sin(el*alpha)
    
    v3k = Vh-25.2*cosl*cosb-245.7*sinl*cosb+276.8*sinb

    return v3k

#################################################################

def Vcmb2Vmod(Vcmb, omegam=0.27):
    
    omegal=1.-omegam
    c=299800.
    z=Vcmb/c
    q0=0.5*(omegam-2.*omegal)
    fmod=1.+0.5*(1.-q0)*z-(1./6)*(1.-q0-3.*q0**2+1.)*z**2
    Vmod=c*z*fmod

    return Vmod
#################################################################

def Vh2Vls(el,b, Vh):
  
    alpha = np.pi / 180.
    cosb = np.cos(b*alpha)
    sinb = np.sin(b*alpha)
    cosl = np.cos(el*alpha)
    sinl = np.sin(el*alpha)
    
    vls = Vh-26.*cosl*cosb+317.*sinl*cosb-8.*sinb

    return vls
#################################################################

In [3]:
catalogs = []
catalogs = ["sh0es"]
catalogs += ["pantheonplus"]
catalogs += ["scolnic"]
catalogs += ["csp1"]
catalogs += ["ganesh"]
catalogs += ["rest"]
catalogs += ["amanullah"]
catalogs += ["jha"]
catalogs += ["prieto"]
catalogs += ["hicken"]
catalogs += ["folatelli"]
catalogs += ["walker"]
catalogs += ["stahl"]
catalogs += ["twins"]
catalogs += ["avelino"]

In [4]:
df = pd.read_csv('./all_cf4_sni1_sample.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df = df.set_index('PGC')
df = df.drop(columns=['1PGC'])
print(len(df))
df.head()

1009


,SNIa,IDT17g,Vcmb,Vmod,DM_av,eDM,DM_pantheonplus,eDM_pantheonplus,DM_scolnic,eDM_scolnic,...,DM_twins,eDM_twins,DM_avelino,eDM_avelino,RA,DE,glon,glat,sgl,sgb
PGC,,,,,,,,,,,,,,,,,,,,,
250,2006sr,0,6914,7042,35.04,0.13,35.07,0.25,35.03,0.20,...,NaN,NaN,NaN,NaN,0.8958,23.2008,108.8693,-38.3606,316.3211,17.6720
282,2008hj,0,10921,11235,36.03,0.11,35.98,0.24,36.00,0.18,...,NaN,NaN,35.97,0.12,1.0064,-11.1742,85.7160,-70.5385,282.0792,8.9703
415,2003it,0,7165,7301,35.12,0.14,35.14,0.25,35.12,0.21,...,NaN,NaN,NaN,NaN,1.4517,27.4491,110.6597,-34.3329,320.8087,17.8681
465,2000dg,0,11092,11415,36.05,0.20,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.5588,8.8878,104.4513,-52.3545,301.9054,14.0485
515,2002hw,0,4796,4857,34.28,0.15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.7062,8.6285,104.5597,-52.6445,301.6830,13.8439


In [5]:
df0 = pd.read_csv('./all_cf4_sni1_sample.csv', delimiter='|')
df0 = df0.rename(columns=lambda x: x.strip())
df0 = df0.set_index('PGC')
df0 = df0[['1PGC']]
print(len(df0))
df0.head()

1009


,1PGC
PGC,
250,250
282,282
415,415
465,616
515,515


In [6]:
df = df0.join(df, how='inner')
print(len(df))
df.head()

1009


,1PGC,SNIa,IDT17g,Vcmb,Vmod,DM_av,eDM,DM_pantheonplus,eDM_pantheonplus,DM_scolnic,...,DM_twins,eDM_twins,DM_avelino,eDM_avelino,RA,DE,glon,glat,sgl,sgb
PGC,,,,,,,,,,,,,,,,,,,,,
250,250,2006sr,0,6914,7042,35.04,0.13,35.07,0.25,35.03,...,NaN,NaN,NaN,NaN,0.8958,23.2008,108.8693,-38.3606,316.3211,17.6720
282,282,2008hj,0,10921,11235,36.03,0.11,35.98,0.24,36.00,...,NaN,NaN,35.97,0.12,1.0064,-11.1742,85.7160,-70.5385,282.0792,8.9703
415,415,2003it,0,7165,7301,35.12,0.14,35.14,0.25,35.12,...,NaN,NaN,NaN,NaN,1.4517,27.4491,110.6597,-34.3329,320.8087,17.8681
465,616,2000dg,0,11092,11415,36.05,0.20,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.5588,8.8878,104.4513,-52.3545,301.9054,14.0485
515,515,2002hw,0,4796,4857,34.28,0.15,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.7062,8.6285,104.5597,-52.6445,301.6830,13.8439


In [7]:
columns = df.columns

for cat in catalogs:
    df["DM_"+cat][df["DM_"+cat]==0] = np.nan
    df["eDM_"+cat][df["eDM_"+cat]==0] = np.nan
    df["eDM_"+cat][df["DM_"+cat]==0] = np.nan
    df["DM_"+cat][df["eDM_"+cat]==0] = np.nan

df

<ipython-input-7-62f1f89f0a54>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DM_"+cat][df["DM_"+cat]==0] = np.nan
<ipython-input-7-62f1f89f0a54>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["eDM_"+cat][df["eDM_"+cat]==0] = np.nan
<ipython-input-7-62f1f89f0a54>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["eDM_"+cat][df["DM_"+cat]==0] = np.nan
<ipython-input-7-62f1f89f0a54>:7: SettingWithCopyWarning: 
A value is try

,1PGC,SNIa,IDT17g,Vcmb,Vmod,DM_av,eDM,DM_pantheonplus,eDM_pantheonplus,DM_scolnic,...,DM_twins,eDM_twins,DM_avelino,eDM_avelino,RA,DE,glon,glat,sgl,sgb
PGC,,,,,,,,,,,,,,,,,,,,,
250,250,2006sr,0,6914,7042,35.04,0.13,35.07,0.25,35.03,...,NaN,NaN,NaN,NaN,0.8958,23.2008,108.8693,-38.3606,316.3211,17.6720
282,282,2008hj,0,10921,11235,36.03,0.11,35.98,0.24,36.00,...,NaN,NaN,35.97,0.12,1.0064,-11.1742,85.7160,-70.5385,282.0792,8.9703
415,415,2003it,0,7165,7301,35.12,0.14,35.14,0.25,35.12,...,NaN,NaN,NaN,NaN,1.4517,27.4491,110.6597,-34.3329,320.8087,17.8681
465,616,2000dg,0,11092,11415,36.05,0.20,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.5588,8.8878,104.4513,-52.3545,301.9054,14.0485
515,515,2002hw,0,4796,4857,34.28,0.15,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.7062,8.6285,104.5597,-52.6445,301.6830,13.8439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003283,9003283,SNF20080626-002,0,7034,7165,35.05,0.10,NaN,NaN,NaN,...,35.05,0.1,NaN,NaN,341.2630,8.9422,78.1768,-42.6849,296.8603,33.5664
9003285,71884,SNF20080725-004,0,16166,16849,36.98,0.10,NaN,NaN,NaN,...,36.98,0.1,NaN,NaN,354.3419,19.9315,100.1204,-39.6369,311.8106,23.1462
9003286,9003286,SNF20080918-002,0,25533,27220,37.95,0.10,NaN,NaN,NaN,...,37.95,0.1,NaN,NaN,340.4511,17.6285,84.1818,-35.3097,307.1052,35.8547


In [8]:
myDict = {}

myDict["sh0es"] = 0
myDict["pantheonplus"] = -0.036
myDict["scolnic"] = -0.007
myDict["csp1"] = 0.001
myDict["ganesh"] = 0.002
myDict["rest"] = -0.001
myDict["amanullah"] = -0.002
myDict["jha"] = 0.008
myDict["prieto"] = -0.001
myDict["hicken"] = 0.002
myDict["folatelli"] = -0.001
myDict["walker"] = -0.037

myDict["stahl"] = -0.001
myDict["twins"] = -0.003
myDict["avelino"] = 0.061

In [9]:
for cat in catalogs:
    df['DM_'+cat] += myDict[cat]
    df['w_'+cat] = (1.-0)/df['eDM_'+cat]**2
    df['w_'+cat] = df['w_'+cat].apply(lambda x: x if x!=0 else np.nan)
    df['e_'+cat] = (1.-0)*df['eDM_'+cat]
    df['e_'+cat] = df['e_'+cat].apply(lambda x: x if x!=0 else np.nan)
    df['xw_'+cat] = (df['DM_'+cat])*df['w_'+cat]

sw = ['w_'+cat for cat in catalogs]
ee = ['e_'+cat for cat in catalogs]
sx = ['xw_'+cat for cat in catalogs]

Err2 = 1./df[sw].sum(axis=1)

df["eDM"] = df[ee].min(axis=1) # np.sqrt(Err2)

df["DM_av"] = df[sx].sum(axis=1)*Err2    


In [10]:
df['av_flag'] = 0*df['DM_av']
df['av_flag'][df['DM_av'].isna()] = 1

<ipython-input-10-d6a2f099f968>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['av_flag'][df['DM_av'].isna()] = 1


In [11]:
for cat in catalogs:
    edm = 'eDM_'+cat
    dm = 'DM_'+cat
    
    ## don't perform the adjsutments on the individual cataloged distances
    df['DM_'+cat] -= myDict[cat]
    
    print(dm, edm)
    
    df[edm] = df.apply(lambda x: x[edm] if x[dm]!=0 else np.nan, axis=1)
    df[dm] = df.apply(lambda x: x[dm] if x[dm]!=0 else np.nan, axis=1)

DM_sh0es eDM_sh0es
DM_pantheonplus eDM_pantheonplus
DM_scolnic eDM_scolnic
DM_csp1 eDM_csp1
DM_ganesh eDM_ganesh
DM_rest eDM_rest
DM_amanullah eDM_amanullah
DM_jha eDM_jha
DM_prieto eDM_prieto
DM_hicken eDM_hicken
DM_folatelli eDM_folatelli
DM_walker eDM_walker
DM_stahl eDM_stahl
DM_twins eDM_twins
DM_avelino eDM_avelino


In [12]:
len(df)

1009

In [13]:
df[df["DM_av"]==0][df.columns[:7]]

,1PGC,SNIa,IDT17g,Vcmb,Vmod,DM_av,eDM
PGC,,,,,,,


In [14]:
df = df[df["DM_av"]!=0]

In [15]:
len(df)

1009

In [16]:
df = df[columns]

In [17]:
df.to_csv("SNIa_CF4_components_bar.csv", sep='|')

In [18]:
df.head()

,1PGC,SNIa,IDT17g,Vcmb,Vmod,DM_av,eDM,DM_pantheonplus,eDM_pantheonplus,DM_scolnic,...,DM_twins,eDM_twins,DM_avelino,eDM_avelino,RA,DE,glon,glat,sgl,sgb
PGC,,,,,,,,,,,,,,,,,,,,,
250,250,2006sr,0,6914,7042,35.050838,0.13,35.07,0.25,35.03,...,NaN,NaN,NaN,NaN,0.8958,23.2008,108.8693,-38.3606,316.3211,17.6720
282,282,2008hj,0,10921,11235,36.037082,0.11,35.98,0.24,36.00,...,NaN,NaN,35.97,0.12,1.0064,-11.1742,85.7160,-70.5385,282.0792,8.9703
415,415,2003it,0,7165,7301,35.134002,0.14,35.14,0.25,35.12,...,NaN,NaN,NaN,NaN,1.4517,27.4491,110.6597,-34.3329,320.8087,17.8681
465,616,2000dg,0,11092,11415,36.048514,0.20,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.5588,8.8878,104.4513,-52.3545,301.9054,14.0485
515,515,2002hw,0,4796,4857,34.284829,0.15,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.7062,8.6285,104.5597,-52.6445,301.6830,13.8439


In [19]:
table   = np.genfromtxt('SNIa_CF4_components_bar.csv' , delimiter='|', 
                        filling_values=-10000, names=True, dtype=None, encoding=None)

colnames = table.dtype.names

## table is a structured array
myTable = {}
for name in table.dtype.names:
    myTable[name] = table[name]
table = myTable
## table is now a dictionary

myTable = Table()

for key in colnames:
    if key in ["SNIa"]:
        myTable.add_column(Column(data=table[key], name=key))
    elif key in ["PGC", "PGC1", "IDT14g", '1PGC']:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int))) 
    elif key in ["Vcmb", "Vmod"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int)))
    elif key in ["RA", "DE", "glon", "glat", "sgl", "sgb"]:
        myTable.add_column(Column(data=table[key], name=key, format='%0.4f'))
    else:
        myTable.add_column(Column(data=table[key], name=key, format='%0.2f'))

## to be used on EDD
myTable.write('SNIa_CF4_components_bar', format='ascii.fixed_width',delimiter='|', bookend=False, overwrite=True)